In [2]:
import sys
import os

src_path = os.environ["src_path"]
sys.path.append(src_path)

In [4]:
from src.components.data_info import *
from datetime import datetime, timedelta

# specify the current time
current_date = pd.to_datetime(datetime.utcnow(), utc=True).floor('h')
print(f'{current_date=}')

# we fetch raw data for the last 28 days, to add redundancy to our data pipeline
fetch_data_to = current_date
fetch_data_from = current_date - timedelta(days=90) #it should only fetch for the last hour but because we dont want the pipline to break in case this data or the provois our was not availble so thats why we added for the entier month
fetch_data_from = pd.Tiemstamp(year=fetch_data_from.year, month=fetch_data_from.month, day=1, tz='UTC')
print(fetch_data_to, fetch_data_from)

current_date=Timestamp('2025-03-12 18:00:00+0000', tz='UTC')


In [5]:
# fetch batch of data from the warehouse
def fetch_batch_raw_data(from_date: datetime, to_date: datetime) -> pd.DataFrame:
    """
    Simulate production data by sampling historical data from 52 weeks ago (i.e. 1 year)
    """
    from_date_ = from_date - timedelta(days=7*52)
    to_date_ = to_date - timedelta(days=7*52)
    print(f'{from_date=}, {to_date=}')

    # download 2 files from website
    demand_values = load_full_data(from_date_, to_date_)
    demand_values = demand_values[(demand_values.date >= from_date_) & (demand_values.date <= to_date_)]

    # shift the data to pretend this is recent data
    demand_values['date'] += timedelta(days=7*52)

    demand_values.sort_values(by=['sub_region_code', 'date'], inplace=True)

    return demand_values


In [ ]:
data = fetch_batch_raw_data(fetch_data_from, fetch_data_to)

In [ ]:
# string to datetime
data['date'] = pd.to_datetime(['date'], utc=True)

# add column with Unix epoch milliseconds
data['seconds'] = data['date'].astype('int64') // 10**6

In [ ]:
data.isna().sum()

In [ ]:
data.tail()

In [7]:
# Install the hopsworks package
%pip install hopsworks

import hopsworks
from src.components import feature_group_config

  Using cached hopsworks-4.1.8-py3-none-any.whl.metadata (11 kB)
  Using cached pyhumps-1.6.1-py3-none-any.whl.metadata (3.7 kB)
  Using cached pyjks-20.0.0-py2.py3-none-any.whl.metadata (1.7 kB)
  Using cached mock-5.2.0-py3-none-any.whl.metadata (3.1 kB)
  Using cached avro-1.11.3.tar.gz (90 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached PyMySQL-1.1.1-py3-none-any.whl.metadata (4.4 kB)
  Using cached retrying-1.3.4-py3-none-any.whl.metadata (6.9 kB)
  Using cached hopsworks_aiomysql-0.2.1-py3-none-any.whl.metadata (11 kB)
  Using cached opensearch_py-2.4.2-py2.py3-none-any.whl.metadata (6.8 kB)
  Using cached protobuf-4.25.6-cp37-abi3-macosx_10_9_universal2.whl.metadata (541 bytes)
  Using cached s3transfer-0.11.4-py3-none-any.whl.metadata (1.7 kB)
  Using cached orderedmultidict-1.0.1-py2.py3-none-any.whl.metadata (1.3 kB)
  Using cached javaobj_py3-0.4.4-py2.py3-none-any.whl.m

TypeError: FeatureGroupConfig.__init__() got an unexpected keyword argument 'event_date'

In [ ]:
hopsworks.__version__

In [8]:
# save predictions to the feature store
from src.components.feature_store_api import get_feature_store
import src.components.feature_group_config as config

# connect to the feature group
feature_group = get_feature_store().get_or_create_feature_group(
    name=config.FEATURE_GROUP_METADATA.name,
    version= config.FEATURE_GROUP_METADATA.version,
    description = "Time-series data at hourly frequency",
    primary_key = ['sub_region_code', 'date'],
    event_time = 'date'
)

TypeError: FeatureGroupConfig.__init__() got an unexpected keyword argument 'event_date'

In [ ]:
feature_group.insert(data, write_options={"wait_for_job": False})